In [ ]:
import os
import shutil
import logging
import numpy as np
from tqdm import tqdm
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import re
from dask.distributed import Client, as_completed
import os.path as osp
import time

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def extract_time_range_from_trc(trc_file):
    with open(trc_file, 'r') as file:
        lines = file.readlines()
    num_frames = int(lines[2].strip().split()[2])
    print(f"Number of frames: {num_frames} for file: {trc_file}")
    data_rate = float(lines[2].strip().split()[0])
    time_end = num_frames / data_rate
    return (0, time_end)

def create_ik_setup_file(template_path, trc_file, output_motion_file, ik_setup_file):
    tree = ET.parse(template_path)
    root = tree.getroot()

    time_range = extract_time_range_from_trc(trc_file)
    time_range_str = f"{time_range[0]} {time_range[1]}"

    time_range_element = root.find('.//time_range')
    time_range_element.text = time_range_str

    marker_file = root.find('.//marker_file')
    marker_file.text = trc_file

    output_motion = root.find('.//output_motion_file')
    output_motion.text = output_motion_file

    tree.write(ik_setup_file)

def clear_file(file_path):
    with open(file_path, 'w') as file:
        file.truncate(0)

import time
import tempfile

def run_inverse_kinematics(scaled_model_file, trc_file, ik_setup_file, output_dir):
    import opensim as osim

    log_file = os.path.join(output_dir, os.path.basename(trc_file).replace('.trc', '_ik.log'))
    ik_output_file = os.path.join(output_dir, os.path.basename(trc_file).replace('.trc', '.mot'))

    # Create a unique temporary directory for each trial to avoid log file conflicts
    with tempfile.TemporaryDirectory() as temp_dir:
        opensim_log = os.path.join(temp_dir, 'opensim.log')

        try:
            # Validate all input files
            if not os.path.exists(scaled_model_file):
                raise FileNotFoundError(f"Scaled model file not found: {scaled_model_file}")
            if not os.path.exists(trc_file):
                raise FileNotFoundError(f"TRC file not found: {trc_file}")
            if not os.path.exists(ik_setup_file):
                raise FileNotFoundError(f"IK setup file not found: {ik_setup_file}")

            # Check if the output already exists to skip processing
            if os.path.exists(ik_output_file):
                logging.info(f"Output file already exists, skipping: {ik_output_file}")
                print(f"\033[93mOutput file already exists, skipping: {ik_output_file}\033[0m")
                return

            # Set up and run the Inverse Kinematics tool
            model = osim.Model(scaled_model_file)
            ik_tool = osim.InverseKinematicsTool(ik_setup_file)
            ik_tool.setModel(model)
            ik_tool.setMarkerDataFileName(trc_file)
            ik_tool.setOutputMotionFileName(ik_output_file)

            # Run the IK tool while ensuring logs are written in the temporary directory
            current_dir = os.getcwd()
            os.chdir(temp_dir)  # Change to temp directory to ensure logs are written here

            logging.debug(f"Running IK Tool with model: {scaled_model_file}, TRC: {trc_file}, Setup: {ik_setup_file}, Output: {ik_output_file}")
            ik_tool.run()

            # Change back to the original directory
            os.chdir(current_dir)

            # Wait for the OpenSim log file to be generated
            start_time = time.time()
            while not os.path.exists(opensim_log) and time.time() - start_time < 10:  # Wait up to 10 seconds
                time.sleep(0.5)

            # Check if the unique log file is successfully generated and copy it to the final log file
            if os.path.exists(opensim_log):
                shutil.copy2(opensim_log, log_file)
                print(f"\033[92mProcessed {trc_file} successfully\033[0m")
            else:
                raise FileNotFoundError(f"OpenSim log file was not generated: {opensim_log}")

            # Ensure the MOT output file is saved
            if not os.path.exists(ik_output_file):
                raise FileNotFoundError(f"Inverse Kinematics output file was not generated: {ik_output_file}")

        except Exception as e:
            print(f"\033[91mError processing {trc_file}: {e}\033[0m")
        finally:
            # Ensure all generated XML files are saved off properly
            if os.path.exists(ik_setup_file):
                shutil.copy2(ik_setup_file, output_dir)

def process_all_subjects(base_dir, subjects, ik_setup_template):
    all_trc_files = []
    all_setup_files = []
    all_scaled_model_files = []
    output_dirs = []

    for subject in subjects:
        subject_formatted = str(subject).zfill(2)

        scaled_model_file = osp.join(base_dir, f'subject_{subject}.osim')
        scaled_model_file = osp.expanduser(scaled_model_file)

        trc_files_dir = osp.join(base_dir, f'P{subject_formatted}', 'raw_marker')
        trc_files_dir = osp.expanduser(trc_files_dir)

        output_dir = osp.join("/home/oliver/workspace/MocapDatasetScripting_REALLAB/temp", f'P{subject_formatted}', 'processed_joint_kinematics')
        output_dir = osp.expanduser(output_dir)

        print(f"Processing subject {subject} with TRC files in: {trc_files_dir}")
        print(f"Output directory: {output_dir}")
        if not os.path.exists(trc_files_dir):
            print(f"\033[91mError: TRC files directory not found: {trc_files_dir}\033[0m")
            continue

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        trc_files = [os.path.join(trc_files_dir, f) for f in os.listdir(trc_files_dir) if f.endswith('.trc')]

        if not trc_files:
            print(f"\033[91mError: No TRC files found in directory: {trc_files_dir}\033[0m")
            continue

        all_trc_files.extend(trc_files)
        all_scaled_model_files.extend([scaled_model_file] * len(trc_files))
        all_setup_files.extend([ik_setup_template] * len(trc_files))
        output_dirs.extend([output_dir] * len(trc_files))

    client = Client(n_workers=9, threads_per_worker=6)
    futures = []
    for scaled_model_file, trc_file, output_dir in zip(all_scaled_model_files, all_trc_files, output_dirs):
        ik_setup_file = os.path.join(output_dir, os.path.basename(trc_file).replace('.trc', '_ik_setup.xml'))
        create_ik_setup_file(ik_setup_template, trc_file, os.path.join(output_dir, os.path.basename(trc_file).replace('.trc', '.mot')), ik_setup_file)
        future = client.submit(run_inverse_kinematics, scaled_model_file, trc_file, ik_setup_file, output_dir)
        futures.append(future)

    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing all subjects in parallel"):
        future.result()

    # Plot subject data after processing all subjects
    subject_logs = get_log_files_by_subject(base_dir, subjects)
    if subject_logs:
        plot_rms_by_subject(subject_logs)

def get_log_files_by_subject(base_dir, subjects):
    subject_logs = {}
    for subject in subjects:
        output_dir = os.path.join(base_dir, f'P{str(subject).zfill(2)}', 'processed_joint_kinematics')
        if not os.path.exists(output_dir):
            print(f"\033[91mWarning: Output directory not found for subject {subject}. Skipping.\033[0m")
            continue
        log_files = [
            os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('_ik.log')
        ]
        subject_logs[subject] = log_files
    return subject_logs

def plot_rms_for_single_trial(log_file):
    rms_values, max_errors = extract_rms_from_log(log_file)
    plt.figure(figsize=(10, 5))
    plt.plot(rms_values)
    plt.title(os.path.basename(log_file))
    plt.xlabel('Frame')
    plt.ylabel('RMS Error')
    plt.ylim([0, 0.10])
    for j, rms in enumerate(rms_values):
        if rms > 0.5:
            plt.plot(j, rms, 'ro')
    plt.tight_layout()
    plt.savefig(log_file.replace('_ik.log', '_rms_plot.png'))  # Save each plot to a separate file
    plt.close()

def plot_rms_by_subject(subject_logs):
    all_rms_values = []
    n_cols = 8

    for subject, logs in subject_logs.items():
        n_trials = len(logs)
        n_rows = (n_trials + n_cols - 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(45, 5 * n_rows))
        axes = axes.flatten()

        for i, log_file in enumerate(logs):
            rms_values, max_errors = extract_rms_from_log(log_file)
            axes[i].plot(rms_values)
            axes[i].set_title(os.path.basename(log_file))
            axes[i].set_xlabel('Frame')
            axes[i].set_ylabel('RMS Error')
            axes[i].set_ylim([0, 0.10])
            for j, rms in enumerate(rms_values):
                if rms > 0.5:
                    axes[i].plot(j, rms, 'ro')

            all_rms_values.append((os.path.basename(log_file), rms_values))

            for marker, errors in max_errors.items():
                average_error = sum(errors) / len(errors)
                print(f"File: {os.path.basename(log_file)}, Marker: {marker}, Count: {len(errors)}, Average Error: {average_error:.6f}")

        for i in range(n_trials, n_rows * n_cols):
            fig.delaxes(axes[i])

        fig.suptitle(f'Subject {subject}')
        plt.tight_layout()
        plt.subplots_adjust(top=0.9)
        plt.savefig(os.path.join(osp.dirname(logs[0]), f'subject_{subject}_rms_plots.png'))  # Save subject-level plots
        plt.close()

def extract_rms_from_log(log_file_path):
    rms_values = []
    max_errors = defaultdict(list)
    with open(log_file_path, 'r') as file:
        for line in file:
            if 'marker error: RMS' in line:
                rms_match = re.search(r'RMS = ([\d.]+)', line)
                max_match = re.search(r'max = ([\d.]+) \((\w+)\)', line)
                if rms_match and max_match:
                    rms_values.append(float(rms_match.group(1)))
                    max_error_value = float(max_match.group(1))
                    max_error_marker = max_match.group(2)
                    max_errors[max_error_marker].append(max_error_value)
    return rms_values, max_errors

if __name__ == "__main__":
    # Set paths to local locations
    base_dir = os.path.expanduser('~/GoogleDrive/sd_datacollection_v4')
    ik_setup_template = os.path.join(base_dir, 'default_ik.xml')

    # Specify subjects and processing options
    subjects = [7, 8, 9, 10, 11, 12, 13]

    # Process all subjects
    process_all_subjects(base_dir, subjects, ik_setup_template)

Number of frames: 5087 for file: /home/oliver/GoogleDrive/sd_datacollection_v4/P07/raw_marker/P07_T1_AS_F.trc
[info] Loaded model Subject-test-scaled from file /home/oliver/GoogleDrive/sd_datacollection_v4/subject_7.osim
Number of frames: 5082 for file: /home/oliver/GoogleDrive/sd_datacollection_v4/P07/raw_marker/P07_T1_AS_N.trc
[warning] Couldn't find file 'r_pelvis.vtp'.
[warning] Couldn't find file 'sacrum.vtp'.
[warning] Couldn't find file 'thoracic12_s.vtp'.
[warning] Couldn't find file 'thoracic11_s.vtp'.
[warning] Couldn't find file 'thoracic10_s.vtp'.
[warning] Couldn't find file 'thoracic9_s.vtp'.
[warning] Couldn't find file 'thoracic8_s.vtp'.
[warning] Couldn't find file 'thoracic7_s.vtp'.
[warning] Couldn't find file 'thoracic6_s.vtp'.
[warning] Couldn't find file 'thoracic5_s.vtp'.
[warning] Couldn't find file 'thoracic4_s.vtp'.
[warning] Couldn't find file 'thoracic3_s.vtp'.
[warning] Couldn't find file 'thoracic2_s.vtp'.
[warning] Couldn't find file 'thoracic1_s.vtp'.
[w